In [2]:
import numpy as np
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta, TH
from sklearn import preprocessing, cross_validation, svm,linear_model
from sklearn.preprocessing import MinMaxScaler



#data = pd.read_csv('Input/tv18_5years.csv', header=0,index_col=False)

data = pd.read_csv('Input/tv18_all.csv', header=0,index_col=False)


data = data[['Date','Open','High','Low','Close','Volume']]
data.fillna(value=-99999, inplace=True)

def calc_last_thursday(year_mon):

    input_date = pd.to_datetime(year_mon + ('-01'))
    cmon = input_date.month
    
    for i in range(1, 6):
        t = input_date + relativedelta(weekday=TH(i))
        if t.month != cmon:
            # since t is exceeded we need last one  which we can get by subtracting -2 since it is already a Thursday.
            t = t + relativedelta(weekday=TH(-2))
            break
    return t.strftime('%Y-%m-%d')


y_label = data[['Date','Close']].loc[1:]
date_calc = y_label['Date'].str[0:7].drop_duplicates()
option_end_Date = []
for i in date_calc:    
    option_end_Date.append(calc_last_thursday(i))   
    
l=[]
for i in range(1,len(data)):
    p = i-1
    if (data['Close'].loc[p] != -99999.000000) or (data['Close'].loc[p] != "null"):   
        oed = 1 if (str(data['Date'].loc[i])) in option_end_Date else 0        
        k= data['Date'].loc[i],data['Open'].loc[p],data['High'].loc[p],data['Low'].loc[p],data['Volume'].loc[p],oed
        l.append(k)    

        
labels= ['Date','Open','High','Low','Volume',"Option EndDate Ind"]
df2 = pd.DataFrame.from_records(l, columns=labels)
df3=y_label.join(df2.set_index('Date'), on='Date')        

In [3]:
# Training and test data preparation
#df3=df1.join(df2.set_index('Date'), on='Date')
split_data = np.array(df3.drop(['Date'], 1))


def cross_validate(split_data):
    n=len(split_data)
    train_start = 0
    train_end = int(np.floor(0.9*n))
    test_start = train_end
    test_end = n
    data_train = split_data[np.arange(train_start, train_end), :]
    data_test = split_data[np.arange(test_start, test_end), :]
    y_train = data_train[:,0]
    y_test = data_test[:,0]

    X_train =data_train[:,1:]
    X_test =data_test[:,1:]
    
    return X_train,y_train,X_test,y_test 


In [4]:
X_train,y_train,X_test,y_test = cross_validate(split_data)
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [37]:

#regression =linear_model.ARDRegression()
#regression.fit(X_train, y_train)
#predicted = regression.predict(X_test)

array([0.15668101, 0.15539413, 0.16298942, 0.04547375, 0.        ])

In [5]:
classifiers = [svm.SVR(),
               linear_model.LinearRegression(),
               linear_model.BayesianRidge(),
               linear_model.ARDRegression(),
               linear_model.TheilSenRegressor()
              ]

for i in classifiers:
    print("running for: %s" % i )
    regression = i
    regression.fit(X_train, y_train)
    predicted = regression.predict(X_test)
    for i in range(len(y_test)-1,len(y_test)-5,-1):
        print(np.round(predicted[i],2),np.round(y_test[i],2))
    print("Mean squared error: %.2f" % np.mean((predicted - y_test) ** 2))
    print('Variance score: %.2f' % regression.score(X_test, y_test))
    
    

running for: SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
45.53 46.65
45.73 46.05
47.11 45.9
47.37 46.7
Mean squared error: 3.36
Variance score: 0.96
running for: LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)
46.27 46.65
46.01 46.05
46.52 45.9
47.61 46.7
Mean squared error: 2.24
Variance score: 0.98
running for: BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, compute_score=False, copy_X=True,
       fit_intercept=True, lambda_1=1e-06, lambda_2=1e-06, n_iter=300,
       normalize=False, tol=0.001, verbose=False)
46.27 46.65
46.02 46.05
46.52 45.9
47.61 46.7
Mean squared error: 2.24
Variance score: 0.98
running for: ARDRegression(alpha_1=1e-06, alpha_2=1e-06, compute_score=False, copy_X=True,
       fit_intercept=True, lambda_1=1e-06, lambda_2=1e-06, n_iter=300,
       normalize=False, threshold_lambda=10000.0, tol=0.001, verbose=False)
46.27 46.65
46.01 46.05
4